In [ ]:
"""
# # Required Packages for the Project RUN ONLY ONCE
# %pip install matplotlib
# %pip install nilearn
# %pip install openpyxl
# %pip install Path
# %pip install seaborn
# %pip install nltools
# %pip install scipy
# %pip install scikit-image
# %pip install nibabel
# %pip install bctpy
# %pip install statsmodels
# %pip install import-ipynb
# %pip install ipynb
# %pip install fitz

"""

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from modules.graphMetrics import GraphMetrics
from modules.helpers import *
from modules.subject import GroupsInfo
from modules.voxel import VoxelCounts
from modules.crossCorrelation import CorrsCorrelations
from modules.createPDF import PDF
import traceback

import logging
logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)
logging.warning('Starting Application....')

In [ ]:
try:
    subjectsData = GroupsInfo.getInstance()
    voxels = VoxelCounts.getInstance()
    graphMetrics = GraphMetrics.getInstance()
    # crossCorr = CorrsCorrelations(subjectsData, voxels, graphMetrics)
except:
    traceback.print_exc()

In [ ]:
# Procedures on voxels data
voxels.calculateVoxelCount()

In [ ]:
# Procedures on graphMetrics data
# graphMetrics.calculateGraphMetrics()
graphMetrics.calAvgAllGroupFNC('values/Weighted-0.15/FNC/groups/')
graphMetrics.calDiffAvgGroupFNC('values/Weighted-0.15/FNC/differences/')

In [ ]:
crossCorrelation = CorrsCorrelations(subjectsData, voxels, graphMetrics)
crossCorrelation.calculateStatsCrossCorrelation()
crossCorrelation.calculateDifferencePvalues()
crossCorrelation.saveCrossCorrelation('values/Weighted-0.15/crossCorrelation/groups/')
crossCorrelation.saveDifferenceCrossCorrelation('values/Weighted-0.15/crossCorrelation/differences/')

In [ ]:
import networkx as nx
gm = dict()
measures = subjectsData.getGlobalGraphMetrics()
groups = list(subjectsData.getGroupDictionary().keys())

for i in groups:
    gm[i] = dict()
    gm[i][measures[0]] = round(computeGlobalEfficiency(numpyMatrix(graphMetrics.avgFNCs[i])),3)
    gm[i][measures[1]] = round(computeCharacteristicPathLength(numpyMatrix(graphMetrics.avgFNCs[i])),3)
    gm[i][measures[2]] = round(computeClusteringCoefficient(numpyMatrix(graphMetrics.avgFNCs[i])),3)

ge_values = pd.DataFrame(gm)
ge_values.to_csv('values/Weighted-0.15/global_values/globalMetrics.txt', sep=',')

groupsMod = list()
for i in range(len(groups)):
    for j in range(i+1, len(groups)):
        groupsMod.append(groups[i]+'/'+groups[j])

t_dict = dict()
p_dict = dict()

for i in measures:
    t_dict[i]=dict()
    p_dict[i] = dict()
    for g1 in range(len(groups)):
        v1 = graphMetrics.graphMetricMap[groups[g1]][i]
        for g2 in range(g1+1, len(groups)):
            v2 = graphMetrics.graphMetricMap[groups[g2]][i]
            
            key1 = groups[g1]+'/'+groups[g2]
            mean_1, mean_2 = gm[groups[g1]][i],gm[groups[g2]][i]
            t_test = two_Sample_t_test(v1, v2, mean_1, mean_2)
            t_dict[i][key1] = t_test[0]
            p_dict[i][key1] = t_test[1]

t_values, p_values = pd.DataFrame(t_dict), pd.DataFrame(p_dict)
t_values.to_csv('values/Weighted-0.15/global_values/t_values.txt', sep=',')
p_values.to_csv('values/Weighted-0.15/global_values/p_values.txt', sep=',')

In [ ]:
import networkx as nx

measures = subjectsData.getComponentGraphMetrics()
groups = list(subjectsData.getGroupDictionary().keys())
indexes = subjectsData.getICNIndexes()
domainList = subjectsData.getICNDomainNames()

def createList(series):
    ans = list()
    for i in range(53):
        ans.append(round(series[i], 5))
    return ans

for i in measures:
    df = prepareDataFrame(indexes, domainList)
    for j in groups:
        if i == "Degree":
            df.loc[len(df.index)] = createList(computeDegree(numpyMatrix(graphMetrics.avgFNCs[j])))

        elif i == "Closeness centrality":
            df.loc[len(df.index)] = createList(computeClosenessCentrality(numpyMatrix(graphMetrics.avgFNCs[j])))

        else:
            df.loc[len(df.index)] = createList(computeParticipationCoefficient(numpyMatrix(graphMetrics.avgFNCs[j])))

    df.index = groups
    # logging.info(df)

    vmin, vmax = 100, -100
    vmin = min( vmin, min(df.min().values))
    vmax = max( vmax, max(df.max().values) )
    logging.info("measure: %s, vmin: %s, vmax: %s", i, vmin, vmax)
    if i == "Closeness centrality": 
        vmin, vmax = 0.55000, 0.66000
    elif i == "Participation coefficient":
        vmin, vmax = 0.89900, 0.95789
    else:
        vmin = min( vmin, min(df.min().values))
        vmax = max( vmax, max(df.max().values) )

    print(list(df.columns))
    prepareHeatmap(
        list(df.columns),
        groups,
        df,
        i,
        'values/Weighted-0.15/global_values/',
        vmin,
        vmax,
        figsize=(8,3)
    )

    diffDt = prepareDataFrame(indexes, domainList)
    rows = df.index.values
    columns = df.columns.values

    newRowList = list()
    for r1 in range(len(rows)):
        for r2 in range(r1+1, len(rows)):

            diffList = list()
            newRowList.append(rows[r1]+'-'+rows[r2])
            for col in columns:
                diffList.append(round(df.at[rows[r1], col] - df.at[rows[r2], col], 3))

            diffDt.loc[len(diffDt.index)] = diffList

    diffDt.index = newRowList
    logging.info("Differences: measure: %s", i)
    logging.info(diffDt)

    vmin = min( vmin, min(diffDt.min().values))
    vmax = max( vmax, max(diffDt.max().values) )
    logging.info("measure: %s, vmin: %s, vmax: %s", i, vmin, vmax)

    greaterVal = max(abs(vmin), abs(vmax))
    vmin, vmax = -1*greaterVal, greaterVal

    prepareHeatmap(
        list(diffDt.columns),
        groups,
        diffDt,
        i+'-Difference',
        'values/Weighted-0.15/global_values/',
        vmin,
        vmax,
        figsize=(8,3)
    )

In [ ]:
getGlobalGraphMetricsForAvgFNCs(graphMetrics, 'values/Weighted-0.15/global_values/', 'Global Metrics')

In [ ]:
"""
100x100 -> 53x53
    69  53  98  99  45 . . . . . . . .
69  0   X   X1  x03 x04 ...............
53  -   x11 x12 x13 x14 . . . . . 
98  -   -   x22 x23 x24 . . . . .
99
45
.
.
.
.

Transpose -> 53x53 FNC-matrix

CrossCorrelation:
entire 69th & 53th -> PearsonCrossCorrelation -> X
> 0.15 or 0 -> Thresholding


Global Metrics:
Entire 53X53 -> 
1 FNC = Y value

CN: [ s_1_X, s_2_X, s_3_X, ..... 900]
AD: [ s_1_X, s_2_X, s_3_X, ..... 200]
MCI: [ s_1_X, s_2_X, s_3_X, ..... 300]

Component Metrics:
Entire 53x53 -> 53 Values 
1 FNC = [ a1, a2, a3, a4,......]

CN:
    69: [s_1_0, s_2_0, ....., 900]
    53: [s_1_1, s_2_1]
    98: [s_1_2, s_2_2]
    99: [s_1_3, s_2_3]
    45: [s_1_4, s_2_4]
AD:
    69: [s_1_0, s_2_0, ....., 200]
    53: [s_1_1, s_2_1]
    98: [s_1_2, s_2_2]
    99: [s_1_3, s_2_3]
    45: [s_1_4, s_2_4]

MCI:
    69: [s_1_0, s_2_0, ....., 300]
    53: [s_1_1, s_2_1]
    98: [s_1_2, s_2_2]
    99: [s_1_3, s_2_3]
    45: [s_1_4, s_2_4]


Cross Correlation:

Voxel:

CN:
    indexes: 
        69: [ s_1_v, s_2_v, ... ,  900]
        53: [ s_1_v, s_2_v, ... ,  900]
        98: [ s_1_v, s_2_v, ... ,  900]
        99: [ s_1_v, s_2_v, ... ,  900]

AD: 
    indexes: 
        69: [ s_1_v, s_2_v, ... ,  900]
        53: [ s_1_v, s_2_v, ... ,  900]
        98: [ s_1_v, s_2_v, ... ,  900]
        99: [ s_1_v, s_2_v, ... ,  900]

Global Metrics:

    1 array of Global Metric of CN Vs ith index of Voxel count

CN:
Global:
GE -> 53 Values
CPL -> 53 Values
CC -> 53 Values

Component:
DE -> 53 values
CCE -> 53 Values
PC -> 53 Values
        0 1 2 3 4 5 6 7 8 ................
    GE
    CPL
    CC
    DE
    CCE
    PC

AD -> 


MCI ->


Difference FNC matrix:

CN:
res - 53x53
sub1 -> 53x53 + res
sub2 -> 53x53 + res

avgFNC matrix - 900 subjects


"""